In [1]:
#This library is used to generate a GUI for any attack type predicted.
import tkinter as tk
#This library is used to create pop a message box.
from tkinter import messagebox
#This library is used to sniff the packet from the network card.
from scapy.all import sniff, IP, TCP, wrpcap
#This is library is used for manipulating arrays.
import numpy as np
#This library is used to load the save CNN model.
from keras.models import load_model
#This library is used performe the statistical analysis on the sniffed packet to extract the required features for 
#prediction.
import statistics
#This is a function from tkinter library helps to create a simple dialog box.
from tkinter import simpledialog
#This is library is used to validate IPV4 address entred by user.
import socket

In [2]:
"""This section of code describes the type of attacks and their label encode attached to them."""

label_encode = [0, 2, 10, 1, 9, 12, 14, 13, 7, 11, 6, 5, 4, 3, 8]
attack_types = ['BENIGN','DDoS', 'PortScan', 'Bot', 'Infiltration',
         'Web Attack � Brute Force', 'Web Attack � XSS',
         'Web Attack � Sql Injection', 'FTP-Patator', 'SSH-Patator',
         'DoS slowloris', 'DoS Slowhttptest', 'DoS Hulk', 'DoS GoldenEye', 'Heartbleed']

result_dict = dict(zip(label_encode, attack_types))
print(result_dict[0])

BENIGN


In [9]:
def consent():
    """This is the function to get user consent to procced with the code and run the IDS.
    _____________________________________________________________________________________
    INPUT : 
    OUTPUT: True or False"""
    while True:
        # Create a tkinter window.
        root = tk.Tk()
        # Hide the main window.
        root.withdraw()  
        
        #Credits: https://www.geeksforgeeks.org/python-tkinter-messagebox-widget/
        # Show a pop-up message box.
        result = messagebox.askquestion("Cookies and Consent!", "This program will monitor network traffic for intrusion detection. Your IP address will be used for this purpose. Do you consent to this?(NOTE:If you do not provide consent this code will terminate without using  any of your data)",type='yesno')
        
        root.destroy()

        # Run the tkinter event loop.
        root.mainloop()

        if result == 'yes':
            # If user consents, break the loop and continue further.
            return True
        elif result == 'no':
            return False
            
def get_ipa():
    # Create a Tkinter window
    root = tk.Tk()
    root.withdraw()  # Hide the main window
    root.geometry("400x150")  # Set the dimensions of the root window
    flag = 0
    while flag == 0:
        #Credits: https://docs.python.org/3/library/dialog.html
        # Prompt the user for input using a dialog box
        user_input = simpledialog.askstring("IP Address", "Please enter the IP address:")
        ip_check = messagebox.askquestion("IP Check!", "Is the entred IP "+ user_input + " correct?",type="yesno")
        try:
            #Credits: https://stackoverflow.com/questions/319279/how-to-validate-ip-address-in-python
            socket.inet_pton(socket.AF_INET, user_input)
            valid_ip_check = True
        except socket.error:
            valid_ip_check = False
        # Display the input provided by the user
        if ip_check == 'yes' and valid_ip_check == True:
            flag = 1
            root.destroy()
            return user_input
        else:
            # Show a pop-up message box.
            messagebox.showinfo("IP Error!", "Wrong IP address or incorrect format detected please try again!")
    root.mainloop()
        
        
def prediction_result(result):
    """This function just prints the predicted result
    _________________________________________________
    INPUT : result
    OUTPUT: print the attack type"""
    
    print("Predicted Attack Type:", result)

def pop_func(result):
    """This function is used to create a alert if any of the following attacks are detected.
    ________________________________________________________________________________________
    INPUT : result
    OUTPUT: message box with alert"""
    attack_list = ['DDoS', 'PortScan', 'Bot', 'Infiltration',
                 'Web Attack � Brute Force', 'Web Attack � XSS',
                 'Web Attack � Sql Injection', 'FTP-Patator', 'SSH-Patator',
                 'DoS slowloris', 'DoS Slowhttptest', 'DoS Hulk', 'DoS GoldenEye', 'Heartbleed']
    # Create a tkinter window.
    root = tk.Tk()
    # Hide the main window.
    root.withdraw()  
    
    if result in attack_list:
        #Credits: https://www.geeksforgeeks.org/python-tkinter-messagebox-widget/
        # Show a pop-up message box.
        messagebox.showinfo("Anomaly Detected!", "The anomaly is "+ result + "!")

    root.destroy()

    # Run the tkinter event loop.
    root.mainloop()
    
def packet_sniffing(packet,ipa):
    """This fucntion extracts the important features from the sinffed packet for further detection.
    _______________________________________________________________________________________________
    INPUT : extracted packet
    OUTPUT: list of important features"""
    #Credits: https://chat.openai.com
    #This list is to append all the captured features from the packet to feed to the model for prediction.
    captured_data = []
    # Extract relevant information from the packet.
    if IP in packet and TCP in packet:        
        #Calculate the flow duration of the packet.
        flow_duration = packet.time - packet[IP].time
        captured_data.append(flow_duration)
        
        # Calculate the total number of forward packets if the source IP is 'ipa'.
        total_fwd_packets = packet['IP'].len if packet['IP'].src == 'ipa' else 0
        captured_data.append(total_fwd_packets)
        
        # Calculate the total number of backward packets if the source IP is not 'ipa'.
        total_bwd_packets = packet['IP'].len if packet['IP'].src != 'ipa' else 0
        captured_data.append(total_bwd_packets)
        
        # Calculate the total length of forward packets if the source IP is 'ipa'.
        total_length_fwd_packets = packet['IP'].len if packet['IP'].src == 'ipa' else 0
        captured_data.append(total_length_fwd_packets)
        
        # Calculate the total length of backward packets if the source IP is not 'ipa'.
        total_length_bwd_packets = packet['IP'].len if packet['IP'].src != 'ipa' else 0
        captured_data.append(total_length_bwd_packets)
        
        # Calculate the flow bytes per second (bps).
        flow_bytes_per_s = total_length_fwd_packets / flow_duration if flow_duration != 0 else 0
        captured_data.append(flow_bytes_per_s)
        
        # Calculate the flow packets per second (pps).
        flow_packets_per_s = 1 / flow_duration if flow_duration != 0 else 0
        captured_data.append(flow_packets_per_s)

        # Calculate the total inter-arrival time for forward packets if the source IP address is 'ipa', 
        #otherwise set it to 0.
        fwd_iat_total = flow_duration if packet['IP'].src == 'ipa' else 0
        captured_data.append(fwd_iat_total)

        # Calculate the total backward inter-arrival time (IAT) if the source IP address is not 'ipa',
        # otherwise set it to 0
        bwd_iat_total = flow_duration if packet['IP'].src != 'ipa' else 0
        captured_data.append(bwd_iat_total)

        # Calculate forward packets per second, ensuring division by zero is handled.
        fwd_packets_per_s = total_fwd_packets / flow_duration if flow_duration != 0 else 0
        captured_data.append(fwd_packets_per_s)
        
        # Calculate backward packets per second, ensuring division by zero is handled
        bwd_packets_per_s = total_bwd_packets / flow_duration if flow_duration != 0 else 0
        captured_data.append(bwd_packets_per_s)
        
        # Calculate the number of FIN flags in the TCP header flags.
        fin_flag_count = (packet['TCP'].flags & 0x02) >> 1
        captured_data.append(fin_flag_count)
        
        # Calculate the number of SYN (Synchronize) flags in the TCP header flags.
        syn_flag_count = (packet['TCP'].flags & 0x02) >> 1
        captured_data.append(syn_flag_count)
        
        # Calculate the number of RST (Reset) flags in the TCP header flags.
        rst_flag_count = (packet['TCP'].flags & 0x04) >> 2
        captured_data.append(rst_flag_count)
        
        # Calculate the number of RST (Reset) flags in the TCP header flags.
        psh_flag_count = (packet['TCP'].flags & 0x08) >> 3
        captured_data.append(psh_flag_count)
        
        # Calculate the number of PSH (Push) flags in the TCP header flags.
        ack_flag_count = (packet['TCP'].flags & 0x10) >> 4
        captured_data.append(ack_flag_count)
        
        # Calculate the number of URG (Urgent) flags in the TCP header flags.
        urg_flag_count = (packet['TCP'].flags & 0x20) >> 5
        captured_data.append(urg_flag_count)
        
        # Calculate the number of CWE (Congestion Window Reduced) flags in the TCP header flags.
        cwe_flag_count = (packet['TCP'].flags & 0x40) >> 6
        captured_data.append(cwe_flag_count)
        
        # Calculate the number of ECE (Explicit Congestion Notification Echo) flags in the TCP header flags.
        ece_flag_count = (packet['TCP'].flags & 0x80) >> 7
        captured_data.append(ece_flag_count)

        # Calculate the average packet size if there are forwarded packets, otherwise set it to 0.
        average_packet_size = total_length_fwd_packets / total_fwd_packets if total_fwd_packets != 0 else 0
        captured_data.append(average_packet_size)

        #Pass the extracted features to model_prediction function for further prediction.
        model_prediction(captured_data)
        
def model_prediction(captured_data):
    """This function is used to predict the result or attack type using the above extraced features and model.
    __________________________________________________________________________________________________________
    INPUT : list of captured feature
    OUTPUT: predict attack type"""
    #Credits: https://chat.openai.com
    # User input
    live_input = np.array(captured_data)

    # Reshape the user input to make it 2D
    live_input = live_input.reshape(1, -1)
    
    # Reshaping the user input for CNN input
    live_input = live_input.reshape(1, live_input.shape[1], 1)

    # Predicting the type of attack
    prediction = loaded_model.predict(live_input)
    pred_result = np.array([np.argmax(prediction)])

    # Convert prediction probabilities to attack types using label encoder
    predicted_attack_type = result_dict[pred_result[0]]

    # Pass the predicted attack type for further processing
    prediction_result(predicted_attack_type)
    
    #Pass the predicted attack type to pop_func to pop an alert if neccessary.
    pop_func(predicted_attack_type)

In [10]:
if __name__ == "__main__":
    #Ask for user permission to proceed with the code.
    user_consent = consent()
    if user_consent == True:
        #Get the IP address of the user laptop or network adaptor.
        ipa = get_ipa()
        # Further program execution continues here after the user gives consent.
        loaded_model = load_model('CNN_IDS_model_with_PCA.h5')
        #Credits: https://thepacketgeek.com/scapy/sniffing-custom-actions/part-1/
        # Start sniffing the network
        sniff(prn=lambda pkt: packet_sniffing(pkt, ipa), store=0, filter="tcp")

    else:
        print("No user consent provided!")
        print("Sorry we can't procced!")

1/1 [==============================] - 0s 364ms/step
Predicted Attack Type: BENIGN
1/1 [==============================] - 0s 16ms/step
Predicted Attack Type: BENIGN
1/1 [==============================] - 0s 16ms/step
Predicted Attack Type: BENIGN
1/1 [==============================] - 0s 16ms/step
Predicted Attack Type: BENIGN
1/1 [==============================] - 0s 16ms/step
Predicted Attack Type: BENIGN
1/1 [==============================] - 0s 28ms/step
Predicted Attack Type: BENIGN
1/1 [==============================] - 0s 16ms/step
Predicted Attack Type: BENIGN
1/1 [==============================] - 0s 23ms/step
Predicted Attack Type: BENIGN
1/1 [==============================] - 0s 16ms/step
Predicted Attack Type: BENIGN
1/1 [==============================] - 0s 16ms/step
Predicted Attack Type: BENIGN
1/1 [==============================] - 0s 22ms/step
Predicted Attack Type: BENIGN
1/1 [==============================] - 0s 16ms/step
Predicted Attack Type: BENIGN
1/1 [==========